In [2]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from ucimlrepo import fetch_ucirepo 
import lime
from lime import lime_tabular

In [3]:
# fetch dataset 
split_size = 0.3
seed = 1234567

wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
X = wine.data.features 
y = wine.data.targets 

n_classes = len(y['class'].unique())
n_feats = X.shape[1]
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = split_size, random_state = seed)

normalizer = Normalizer().fit(x_train)
encoder = OneHotEncoder().fit(y_train)

x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)
y_train = encoder.transform(y_train)
y_test  = encoder.transform(y_test)

In [4]:
n_classes, n_feats

(3, 13)

In [5]:
def evaluate_model(model, x, y):
    pred = model.predict(x)
    return accuracy_score(y, pred)

In [6]:
model = MLPClassifier((n_feats, 256, n_classes),
                      activation = 'relu',
                      learning_rate = 'adaptive', 
                      learning_rate_init=0.029, 
                      max_iter = 1000,
                      random_state=3)
model.fit(x_train, y_train)

acc_train = evaluate_model(model, x_train, y_train)
acc_test  = evaluate_model(model, x_test, y_test)

print(acc_train, acc_test)

0.5967741935483871 0.6296296296296297


In [7]:
x_test[0].shape

(13,)

In [8]:
model.predict_proba(x_test[0][None, :])

array([[0.58010298, 0.08305055, 0.30143897]])

In [9]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data = x_train,
    mode = 'classification'
)

exp = explainer.explain_instance(x_test[1], model.predict_proba, num_features = n_feats)
exp.local_exp

c:\ProgramData\Anaconda3\envs\torchnew\Lib\site-packages\lime\lime_tabular.py:372: UserWarning: 
                    Prediction probabilties do not sum to 1, and
                    thus does not constitute a probability space.
                    Check that you classifier outputs probabilities
                    (Not log probabilities, or actual class predictions).
                    
  warnings.warn("""


{1: [(9, -0.17519128813176663),
  (3, 0.13046890719713095),
  (8, 0.08373988277210624),
  (1, 0.06959551144327851),
  (10, -0.044298332563636036),
  (4, -0.039539361982557454),
  (6, 0.03625815822408123),
  (2, -0.02122096046158452),
  (12, -0.017440225062098837),
  (5, -0.016577653516246144),
  (7, -0.011555468068045539),
  (11, -0.004529974451325677),
  (0, 0.004214644864743017)]}

In [10]:
sum([abs(x[1]) for x in exp.local_exp[1]])

0.6546303687386007

In [11]:
# TODO
# add bayesian hyperparameter optimization for finding optimal number of components in gaussian mixture model (https://www.run.ai/guides/hyperparameter-tuning/bayesian-hyperparameter-optimization)
# do sampling and hyperparameter exstimation
# https://www.mdpi.com/2673-2688/4/2/23#:~:text=Post%2Dhoc%20explanation%20methods%20can,of%20underlying%20black%2Dbox%20model.
# https://arxiv.org/pdf/1907.03039.pdf
# https://arxiv.org/pdf/2101.07685.pdf
# https://proceedings.neurips.cc/paper_files/paper/2020/file/24aef8cb3281a2422a59b51659f1ad2e-Paper.pdf
# https://www.analyticsvidhya.com/blog/2018/11/reinforcement-learning-introduction-monte-carlo-learning-openai-gym/
# https://faculty.washington.edu/yenchic/21Sp_stat542/Lec3_Mixture.pdf
# https://machinelearningmastery.com/probabilistic-model-selection-measures/
# https://www.run.ai/guides/hyperparameter-tuning/bayesian-hyperparameter-optimization
# https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html#sklearn.mixture.BayesianGaussianMixture
# https://stats.stackexchange.com/questions/226834/sampling-from-a-mixture-of-two-gamma-distributions/226837#226837

# https://github.com/marcotcr/lime/blob/master/lime/explanation.py
# https://lime-ml.readthedocs.io/en/latest/lime.html#
# https://www.analyticsvidhya.com/blog/2022/07/everything-you-need-to-know-about-lime/
# https://github.com/msetzu/glocalx
# https://drive.google.com/file/d/1gW7V4FyUj9vajrGbIkoa7mbbZn4xEbD6/view